In [1]:
import requests
from bs4 import BeautifulSoup as soup
import os, time
import json,re
import random

In [2]:
main_url = "https://stats.espncricinfo.com/ci/engine/stats/index.html?class=11;filter=advanced;orderby=runs;size=150;team=8;template=results;type=batting"

In [3]:
response = requests.get(
    url = main_url,
)

main_soup = soup(response.content, 'html.parser')

In [4]:
cricketers_list_raw = main_soup.find_all('tr', "data1")

In [5]:
cricketers_list = []
for i in range(0,100):
    for x in cricketers_list_raw[i].find_all("a", "data-link"):
        cricketers_list.append({"name": x.getText(), "url": x.get("href")})
    

In [6]:
# cricketers_list

In [7]:
base_url_player = "https://stats.espncricinfo.com/"

In [8]:
# Filter dictionary keeping necessary fields
def filter_dict(di, keys):
    if not isinstance(di, (dict, list)):
        return di
    if isinstance(di, list):
        return [filter_dict(value, keys) for value in di]
    return {key: filter_dict(value, keys) for key, value in di.items() if key in keys or isinstance(value, (dict, list))}


In [9]:
player_data_keys = ["Full Name", "Born", "Age", "Batting Style", "Bowling Style", "Playing Role"]
def extract_player_data(soup):
    player_card = soup.find_all('div', "player_overview-grid")
    player_details_raw = player_card[0].find_all('div')
    player_details = {}
    for detail in player_details_raw:
        if(detail.p.getText() == "Fielding Position"):
            continue
        player_details[detail.p.getText()] = detail.h5.getText()
    if(not "Bowling Style" in player_details):
        player_details["Bowling Style"] = "N/A"
    if(not "Playing Role" in player_details):
        player_details["Playing Role"] = "N/A"
    # print(player_details["Full Name"])
    return filter_dict(player_details, player_data_keys)
    

In [32]:
def extract_odi_data(soup):
    standings_card_raw = soup.find_all("div", "overflow-hidden")
    tables_raw_check = standings_card_raw[2].find_all("h5")
    # print(tables_raw_check)
    
    # Finding Batting and Bowling tables and extracting the table
    for i in range (2):
        print(tables_raw_check[i].getText())
        if(tables_raw_check[i].getText()=="Batting & Fielding"):
            batting_table_raw = tables_raw_check[i].find_next("table", "table standings-widget-table text-center mb-0 border-bottom")
            # print(batting_table_raw.prettify())
        else:
            bowling_table_raw = tables_raw_check[i].find_next("table", "table standings-widget-table text-center mb-0 border-bottom")

    player_values = {}
    # Batting Values extraction
    # Finding ODI row and extracting the row
    batting_table_rows_raw = batting_table_raw.find("tbody").find_all("tr")
    for row in batting_table_rows_raw:
        # print(row.prettify())
        if (row.find("span").getText()=="ODI"):
            batting_table_odi_row = row
            break
        else:
            continue
    batting_values_raw = batting_table_odi_row.find_all("span")
    player_values["Matches"] = batting_values_raw[1].getText()
    player_values["Runs"] = batting_values_raw[4].getText()
    player_values["Average"] = batting_values_raw[6].getText()
    player_values["Half-centuries"] = batting_values_raw[10].getText()
    player_values["Centuries"] = batting_values_raw[9].getText()

    # Bowling values extraction
    # Finding ODI row and extracting the row
    bowling_table_rows_raw = bowling_table_raw.find("tbody").find_all("tr")
    for row in bowling_table_rows_raw:
        if (row.find("span").getText()=="ODI"):
            bowling_table_odi_row = row
            break
        else:
            continue
    bowling_values_raw = bowling_table_odi_row.find_all("span")
    player_values["wickets"] =bowling_values_raw[5].getText() if(bowling_values_raw[5].getText()!="-") else "N/A"
    player_values["economy"] =bowling_values_raw[9].getText() if(bowling_values_raw[9].getText()!="-") else "N/A"
    print(player_values)
    return player_values
    

In [33]:
cricketers = []
for i in range(20,21):
    # print(i)
    player_url = base_url_player + cricketers_list[i]["url"]
    response = requests.get(
    url = player_url,
    )
    player_soup = soup(response.content, 'html.parser')
    player_data = extract_player_data(player_soup) | extract_odi_data(player_soup)
    cricketers.append(player_data)

Bowling
Batting & Fielding
<tr class="bg-gray-100 fix-first-child-color">
 <td class="border-right pl-2 pr-2 stats_table-td-font pl-3 pr-0 font-weight-bold w-15 text-nowrap">
  <span class="out-padding">
   Test
  </span>
 </td>
 <td class="border-right pl-2 pr-2 stats_table-td-font w-8-5 text-nowrap text-right">
  <span class="out-padding">
   111
  </span>
 </td>
 <td class="border-right pl-2 pr-2 stats_table-td-font w-8-5 text-nowrap text-right">
  <span class="out-padding">
   194
  </span>
 </td>
 <td class="border-right pl-2 pr-2 stats_table-td-font w-8-5 text-nowrap text-right">
  <span class="out-padding">
   23438
  </span>
 </td>
 <td class="border-right pl-2 pr-2 stats_table-td-font w-8-5 text-nowrap text-right">
  <span class="out-padding">
   10501
  </span>
 </td>
 <td class="border-right pl-2 pr-2 stats_table-td-font w-8-5 text-nowrap text-right benton-bold">
  <span class="out-padding">
   355
  </span>
 </td>
 <td class="border-right pl-2 pr-2 stats_table-td-font w-8-5

In [240]:
# print(cricketers)

In [241]:
import tabulate
dataset = cricketers
header = dataset[0].keys()
rows =  [x.values() for x in dataset]
# print (tabulate.tabulate(rows, header))